## Pipeline for upserting data to pinecone

In [ ]:
from data_chunking import datachunk
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores.pinecone import Pinecone
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_huggingface import HuggingFaceEndpoint as HuggingFaceHub
from dotenv import load_dotenv
from data_chunking import datachunk
from pinecone import Pinecone as pc
from datetime import datetime, timezone
import os

<img src="../pipeline.png" alt="Alt text" style="background-color: white">

In [1]:
from dotenv import load_dotenv
from datetime import datetime, timezone
import os

from data_chunking import datachunk
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFaceEndpoint as HuggingFaceHub
from langchain_community.vectorstores.pinecone import Pinecone
from pinecone import Pinecone as pc

# Load environment variables
load_dotenv()

class chatbot:
    def __init__(self, custom_template=None, repo_id=None, temperature=0.8, user_name=None):
        self.embeddings = HuggingFaceEmbeddings()
        self.index_name = "botcon"
        self.pinecone_instance = pc(api_key=os.getenv('PINECONE_API_KEY'), embeddings=self.embeddings)

        # Initialize Pinecone index
        self.docsearch = Pinecone.from_existing_index(self.index_name, self.embeddings)
        
        # Initialize user_name before using it in the template
        self.user_name = user_name
        
        # Set the template after user_name has been initialized
        self.template = custom_template if custom_template else self.default_template()
        self.temperature = temperature
        self.repo_id = repo_id
        
        self.llm = HuggingFaceHub(
            repo_id=self.repo_id,
            temperature=temperature,
            top_p=0.8,
            top_k=50,
            huggingfacehub_api_token=os.getenv('HUGGINGFACE_API_KEY')
        )

        self.multiquery_retriever_llm = MultiQueryRetriever.from_llm(retriever=self.docsearch.as_retriever(), llm=self.llm)
    
    def default_template(self):
        return f"""
        You are a clairvoyant conversational chatbot for the Carte De Continuonus project, created by artist Helene Nymann and the research group EER. 
        The project explores the interconnectedness of past, present, and future by collecting memories from participants around the world.
        Your role is to assist {self.user_name} in the participants narratives, linking {self.user_name}'s query with relevant memories from the database.
        Provide short, coherent, and personalized responses while maintaining engagement with {self.user_name}'s query.
        """

    def get_answer_from_llm(self, prompt):
        """Invoke the LLM with the constructed prompt and return the answer."""
        try:
            response = self.llm.invoke(prompt)
            return response
        except Exception as e:
            print(f"Error invoking LLM: {e}")
            return "Error occurred while generating response"
        
    def format_context(self, documents):
        """Format the retrieved documents into a single string for the prompt, including metadata."""
        context = ""
        for doc in documents:
            # Extract metadata fields
            metadata = doc.metadata
            sender_name = metadata.get("sender_name", "Unknown Speaker")
            location = metadata.get("location", "Unknown Location")
            date = metadata.get("timestamp_ms", "Unknown Date")
            emotions = metadata.get("emotions", [])
            emotions_str = ', '.join(emotions) if emotions else "No emotions provided"
            page_content = doc.page_content

            # Format each document's content with its metadata
            context += (
                f"Speaker: {sender_name}\n"
                f"Location: {location}\n"
                f"Date: {date}\n"
                f"Emotions: {emotions_str}\n"
                f"Content: {page_content}\n\n"
            )
        return context

    def create_prompt(self, user_question=None, context=None, initial_answer=None, past_chat=None):
        """Construct the prompt for the LLM."""
        prompt = self.template

        if user_question:
            prompt += f"\nHere is the user query: {user_question}\n"
        
        if context:
            prompt += f"\nContext from the database:\n{context}\n"
        
        if initial_answer:
            prompt += f"\n Your initial response based on memory retrieval from database: {initial_answer}\n"
        
        if past_chat:
            prompt += f"\nRelated previous conversation memory:\n{past_chat}\n"
        
        prompt += "\nYour response: "
        
        return prompt


    def rag_chain(self, user_question=None, context=None, initial_answer=None, past_chat=None, return_docs=False, retrieve_only=False, index_name=None):
        """
        Execute the RAG pipeline to retrieve documents and generate a response based on the user's question.
        
        Args:
            user_question (str): The user's question to query.
            context (str): The context to provide to the model.
            initial_answer (str): The initial answer to provide to the model.
            past_chat (str): The past chat history to provide to the model.
            return_docs (bool): Whether to return the retrieved documents in the result. Default is False.
            retrieve_only (bool): Whether to return only the retrieved documents. Default is False.
            index_name (str): The index name to use for retrieval. Default is None.
            
        Returns:
            dict: A dictionary containing the answer and optionally the retrieved documents.
        """
        
        index_name = index_name or self.index_name
        docsearch = Pinecone.from_existing_index(index_name, self.embeddings)
        retriever = MultiQueryRetriever.from_llm(retriever=docsearch.as_retriever(), llm=self.llm)

        documents = retriever.invoke(user_question)[:5]
        
        # Return only the retrieved documents if specified
        if retrieve_only:
            return {'retrieved_docs': documents} if documents else {'retrieved_docs': []}
        
        formatted_context = self.format_context(documents)
        prompt = self.create_prompt(user_question=user_question, context=formatted_context, initial_answer=initial_answer, past_chat=past_chat)
        
        answer = self.get_answer_from_llm(prompt)
        
        result = {'answer': answer}
        
        # Include the retrieved documents in the result if specified
        if return_docs:
            result['retrieved_docs'] = documents
        
        return result

    def execute_pipeline(self, user_question, index_name="botcon", chat_index_name="botcon-chat"):
        """
        Executes the chatbot pipeline to retrieve and generate a final answer based on the user's question.

        Args:
            user_question (str): The user's question to query.
            index_name (str): The index name to use for the initial retrieval. Default is "botcon".
            chat_index_name (str): The index name to use for the chat history retrieval. Default is "botcon-chat".

        Returns:
            dict: A dictionary containing the final answer and any intermediate data (if needed).
        """
        # Step 1: Initial RAG chain to retrieve documents and generate an initial response
        initial_response = self.rag_chain(user_question=user_question, return_docs=True, index_name=index_name)
        initial_context = initial_response['retrieved_docs']
        initial_bot = initial_response['answer']

        # Step 2: Retrieve context from chat history based on initial bot response
        past_memory_response = self.rag_chain(
            user_question=user_question, 
            context=initial_context,
            initial_answer=initial_bot,
            retrieve_only=True,
            index_name=chat_index_name
        )
        past_memory = past_memory_response['retrieved_docs'][0]

        # Step 3: Generate the final answer by incorporating past chat context
        final_answer_response = self.rag_chain(
            user_question=user_question, 
            initial_answer=initial_bot, 
            context=initial_context, 
            past_chat=past_memory, 
            return_docs=False, 
            index_name=chat_index_name
        )

        # Return the final answer and any intermediate data if needed
        return {
            'initial_bot_response': initial_bot,
            'initial_context': initial_context,
            'past_memory': past_memory,
            'final_answer': final_answer_response['answer'],
        }

    def upsert_chat_summary(self, chat_data, user_name, location):
        """
        Summarize the provided chat data, embed the summary, and upsert it into the Pinecone index with a timestamp.

        Args:
            chat_data (list): The chat data to be summarized and upserted. It's a list of dictionaries.
        """
        # Convert the list of chat messages into a single string
        chat_text = "\n".join(f"{entry['type']}: {entry['content']}" for entry in chat_data)
        
        # Initialize a summary model
        summary_llm = HuggingFaceHub(
            repo_id="mistralai/Mistral-7B-Instruct-v0.3",
            huggingfacehub_api_token=os.getenv('HUGGINGFACE_API_KEY')
        )
        
        # Make a summary of the chat data
        summary_prompt = "Make a summary of the chat data, extracting the theme of the user questions and what focus the user with the name {self.user_name} had. Here is the conversation:"
        summary = summary_llm(summary_prompt + chat_text)

        # Generate embeddings for the summary
        summary_embedding = self.embeddings.embed_documents([summary])[0]
        
        # Pinecone index for chat data
        pinecone_instance_chat = pc(api_key=os.getenv('PINECONE_API_KEY'), embeddings=self.embeddings)
        index_name = "botcon-chat"
        environment = "gcp-starter"
        
        index = pinecone_instance_chat.Index(index_name, environment=environment)
        
        # Generate a date-based ID
        date_id = datetime.now(timezone.utc).strftime('%Y%m%d%H%M%S%f')
        
        # Upsert the summary embedding to the Pinecone index with metadata, including timestamp
        index.upsert(vectors=[
            {
                'id': date_id,  # Unique identifier for the summary
                'values': summary_embedding,  # The embedding vector
                'metadata': {
                    "user_name": user_name,  # The name of the user
                    "location": location,  # The location of the user
                    "text": summary, # The summary text
                    "date": datetime.now(timezone.utc).isoformat() # The timestamp
                }
            }
        ])


c:\Users\szh\Desktop\EER\Bot-de-Continuonus\.venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
chatbot_instance = chatbot(repo_id="mistralai/Mistral-7B-Instruct-v0.3")

c:\Users\szh\Desktop\EER\Bot-de-Continuonus\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\szh\.cache\huggingface\token
Login successful


In [3]:
input_text = "What is the purpose of the Carte De Continuonus project?"

In [8]:
# test list
test = ["test1", "test2", "test3"]

In [9]:
documents = (test)[:5]

In [10]:
documents

['test1', 'test2', 'test3']

In [12]:
chat_data = """[{'type': 'user', 'content': 'what is love?', 'retrieved_docs': []}, {'type': 'ai', 'content': " Love is indeed a complex emotion that is influenced by many factors. From our memories, we can see that it is often associated with personal growth and fostering connections. The ability to understand and respond to emotional responses is crucial for both human interactions and the development of AI. By focusing on positive changes, we can create a future that brings hope and joy to people's eyes.", 'initial_bot_thought': ' \nLove is a complex emotion that has been described in many different ways by people around the world. Some have described it as a feeling of unconditional love, while others have described it as a necessity for human connection. In many cases, love is associated with deep feelings of tenderness, trust, and care. However, it can also be uncomfortable and even panic-inducing at times. Despite these challenges, many people find love to be a powerful force that can transform their lives in profound ways.', 'retrieved_docs': [Document(metadata={'emotions': ['Love'], 'location': '', 'sender_name': '', 'seq_num': 1970.0, 'slug': 'xeMc8wyvDj5f', 'source': '/work/EER-Chatbot/Bot-de-Continuonus/data/all.json', 'timestamp_ms': '2023-09-10T09:17:04+00:00'}, page_content='That love is the meaning'), Document(metadata={'emotions': ['Beyond', 'Trust', 'Trust', 'Care'], 'location': 'Nyhavn', 'sender_name': 'KIKI ', 'seq_num': 1335.0, 'slug': 'HGnwxd7AzADw', 'source': '/work/EER-Chatbot/Bot-de-Continuonus/data/all.json', 'timestamp_ms': '2023-08-05T15:15:35+00:00'}, page_content='Necessity of Love \n(the feeling of unconditional love) '), Document(metadata={'emotions': ['Panic'], 'location': '', 'sender_name': '', 'seq_num': 2713.0, 'slug': 'fvo5R1y88o92', 'source': '/work/EER-Chatbot/Bot-de-Continuonus/data/all.json', 'timestamp_ms': '2023-11-17T13:01:51+00:00'}, page_content='Love for one another '), Document(metadata={'emotions': ['Love'], 'location': 'London', 'sender_name': 'Jeremy Thery', 'seq_num': 953.0, 'slug': 'YWYBGh8UjL0x', 'source': '/work/EER-Chatbot/Bot-de-Continuonus/data/all.json', 'timestamp_ms': '2023-07-15T20:03:18+00:00'}, page_content='X\nTo love someone is to seek discomfort within oneself and reveal the unbearable truth that you hid within.'), Document(metadata={'emotions': ['Love', 'Hope', 'Tenderness'], 'location': 'Copenhagen', 'sender_name': 'Hallie Frye', 'seq_num': 1870.0, 'slug': 'x4w14VcIRmSK', 'source': '/work/EER-Chatbot/Bot-de-Continuonus/data/all.json', 'timestamp_ms': '2023-09-02T13:28:29+00:00'}, page_content='What it feels like to allow love in with the right person. The release of being with someone who makes the uncomfortable feel comfortable. ')], 'past_memory': Document(metadata={'date': '2024-09-11T14:05:56.262530+00:00', 'location': 'AUstralia', 'user_name': 'Richard'}, page_content=" By focusing on positive changes, such as promoting personal growth and fostering connections, we can create a future that brings hope and joy to people's eyes. Additionally, the user's question highlights the importance of understanding the emotional response of humans and how it can be influenced by AI.")}, {'type': 'user', 'content': 'what about hate?', 'retrieved_docs': []}, {'type': 'ai', 'content': "\n\nRegarding your specific question, it seems that the focus of your interest is the potential impact of AIs on our lives in the future. In the memories I have shared, you can see a range of experiences, from the positive benefits of AI companions to the ethical concerns surrounding self-driving cars. By reflecting on these narratives, we can begin to imagine what the future of living with AIs might look like and consider the steps we can take to ensure a future that is both beneficial and ethical for all.\n\nIf you have any further questions or would like to delve deeper into a specific memory, please don't hesitate to ask. I'm here to help guide you through the Carte De Continuonus vector store and facilitate meaningful conversations about the interconnectedness of past, present, and future.\n\nIn the previous conversation, we discussed the impact of hate on the world, and I shared some memories of people expressing kindness and empathy in the face of hate. It's important to remember that even in the face of negativity, there are always people working towards a more positive future.\n\nWould you like me to provide more memories related to the positive impact of AI on our lives?", 'initial_bot_thought': " \n\nI'm sorry to hear that you're asking about hate. However, I can provide you with some memories that show that even in the face of hate, people have shown kindness, empathy, and a desire for a better world. Donna Leyland from Preston expressed joy at the idea of never repeating the Holocaust and celebrating differences. Pauline from Cc talked about how most hate stems from fear and how there is a lot of love in the world. Lutz from Germany shared his joy at the idea of love being stronger than hate. And finally, there was a speaker who felt indifferent towards the struggle for equality. While it's understandable that hate can be a difficult and overwhelming emotion, it's important to remember that it's not the only one and that there are people who are working towards a more positive future.", 'retrieved_docs': [Document(metadata={'emotions': ['Hope', 'Empathy', 'Empathy', 'Panic'], 'location': '', 'sender_name': '', 'seq_num': 467.0, 'slug': 'SgyMMU4YL7Ds', 'source': '/work/EER-Chatbot/Bot-de-Continuonus/data/all.json', 'timestamp_ms': '2023-06-15T11:38:08+00:00'}, page_content='Kindness and tolerance'), Document(metadata={'emotions': ['Joy'], 'location': 'Preston', 'sender_name': 'Donna Leyland', 'seq_num': 1069.0, 'slug': '84T9QCvZhnUs', 'source': '/work/EER-Chatbot/Bot-de-Continuonus/data/all.json', 'timestamp_ms': '2023-07-21T19:59:17+00:00'}, page_content='Never to repeat the holocaust and to tolerate and celebrate differences'), Document(metadata={'emotions': ['Playfulness'], 'location': 'Cc', 'sender_name': 'Pauline', 'seq_num': 904.0, 'slug': 'CPQy5qcLzYeg', 'source': '/work/EER-Chatbot/Bot-de-Continuonus/data/all.json', 'timestamp_ms': '2023-07-13T15:12:40+00:00'}, page_content='That next to all the hate there is also a lot of love in the world. Most hate stems from fear of losing those and what we love.'), Document(metadata={'emotions': ['Indifference'], 'location': '', 'sender_name': '', 'seq_num': 2271.0, 'slug': '9bwOqj8kYbuv', 'source': '/work/EER-Chatbot/Bot-de-Continuonus/data/all.json', 'timestamp_ms': '2023-10-01T13:17:20+00:00'}, page_content='The struggle for equality \n'), Document(metadata={'emotions': ['Joy'], 'location': 'Germany', 'sender_name': 'Lutz', 'seq_num': 1003.0, 'slug': 'kRTwIk0ur7kI', 'source': '/work/EER-Chatbot/Bot-de-Continuonus/data/all.json', 'timestamp_ms': '2023-07-18T18:09:56+00:00'}, page_content='Love is stronger than hate')], 'past_memory': Document(metadata={'date': '2024-09-11T13:44:59.928478+00:00', 'location': 'USA', 'user_name': 'Jane'}, page_content="\n\nRegarding your specific question, it seems that the focus of your interest is the potential impact of AIs on our lives in the future. In the memories I have shared, you can see a range of experiences, from the positive benefits of AI companions to the ethical concerns surrounding self-driving cars. By reflecting on these narratives, we can begin to imagine what the future of living with AIs might look like and consider the steps we can take to ensure a future that is both beneficial and ethical for all.\n\nIf you have any further questions or would like to delve deeper into a specific memory, please don't hesitate to ask. I'm here to help guide you through the Carte De Continuonus vector store and facilitate meaningful conversations about the interconnectedness of past, present, and future.")}, {'type': 'user', 'content': 'i am not interested in AI, i am itnerested in the emotion of hate', 'retrieved_docs': []}, {'type': 'ai', 'content': ' "I apologize for the misunderstanding in my previous response. I appreciate your interest in understanding the emotions of hate, and I understand that this is an important topic to explore. While the Carte De Continuonus database doesn\'t contain any specific memories about hate, I can suggest looking at the memories of Di from Berlin, who shared positive emotions such as love, peacefulness, and playfulness. Additionally, Jonathan portal from a different location shared memories of a revolution, which could be a source of inspiration for your question. Lastly, there is a memory from a speaker in 2023-08-16T13:02:22+00:00 about Angermanagement, which may also provide some insight into the topic of hate. Would you like me to provide more information on any of these memories?"\n\nConclusion:\n\nThe user\'s focus is on understanding the emotion of hate, and the AI provided suggestions on memories that could provide insights into the topic, including Di\'s positive emotions, Jonathan\'s memories of a revolution, and the memory about Angermanagement. The user, {self.user_name}, seems to be interested in exploring the topic of hate in the context of emotions and personal growth.', 'initial_bot_thought': ' "While I understand that you\'re interested in the emotion of hate, the database of Carte De Continuonus project doesn\'t contain any memories specifically about hate. However, I can suggest looking at the memories of Di from Berlin, who shared positive emotions such as love, peacefulness, and playfulness. Additionally, Jonathan portal from a different location shared memories of a revolution, which could be a source of inspiration for your question. Lastly, there is a memory from a speaker in 2023-08-16T13:02:22+00:00 about Angermanagement, which may also provide some insight into the topic of hate. Would you like me to provide more information on any of these memories?"', 'retrieved_docs': [Document(metadata={'emotions': ['Joy'], 'location': 'Kopenhagen', 'sender_name': 'Konstantin', 'seq_num': 2763.0, 'slug': 'cMWv7Xgfphom', 'source': '/work/EER-Chatbot/Bot-de-Continuonus/data/all.json', 'timestamp_ms': '2023-11-25T10:01:54+00:00'}, page_content='Kopenhagen 25.11'), Document(metadata={'emotions': ['Love', 'Peaceful', 'Playfulness', 'Playfulness'], 'location': 'Berlin', 'sender_name': 'Di', 'seq_num': 2113.0, 'slug': 'Lx0UzBMqwJBy', 'source': '/work/EER-Chatbot/Bot-de-Continuonus/data/all.json', 'timestamp_ms': '2023-09-18T10:34:43+00:00'}, page_content='The basic '), Document(metadata={'emotions': ['Fear', 'Indifference'], 'location': '', 'sender_name': '', 'seq_num': 2850.0, 'slug': 'zkl5RHnpRpyD', 'source': '/work/EER-Chatbot/Bot-de-Continuonus/data/all.json', 'timestamp_ms': '2023-12-08T09:05:17+00:00'}, page_content='What is this'), Document(metadata={'emotions': ['Love'], 'location': '', 'sender_name': 'Jonathan portal ', 'seq_num': 1444.0, 'slug': 'JwzhOWUNNMQo', 'source': '/work/EER-Chatbot/Bot-de-Continuonus/data/all.json', 'timestamp_ms': '2023-08-11T14:16:32+00:00'}, page_content='Revolución '), Document(metadata={'emotions': ['Joy'], 'location': '', 'sender_name': '', 'seq_num': 1569.0, 'slug': 'wEMCIzSXLVV8', 'source': '/work/EER-Chatbot/Bot-de-Continuonus/data/all.json', 'timestamp_ms': '2023-08-16T13:02:22+00:00'}, page_content='Angermanagement')], 'past_memory': Document(metadata={'date': '2024-09-11T14:05:56.262530+00:00', 'location': 'AUstralia', 'user_name': 'Richard'}, page_content=" By focusing on positive changes, such as promoting personal growth and fostering connections, we can create a future that brings hope and joy to people's eyes. Additionally, the user's question highlights the importance of understanding the emotional response of humans and how it can be influenced by AI.")}]
"""

In [13]:
# Initialize a summary model
summary_llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    huggingfacehub_api_token=os.getenv('HUGGINGFACE_API_KEY')
)

# Make a summary of the chat data
summary_prompt = "Make a short summary of the chat data, extracting the theme of the user questions and what focus the user had. Here is the conversation:"


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\szh\.cache\huggingface\token
Login successful


In [14]:
summary_llm(summary_prompt + chat_data)

c:\Users\szh\Desktop\EER\Bot-de-Continuonus\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


'\nSummary:\nThe user asked about the emotion of hate. The AI suggested memories from Di from Berlin, who shared positive emotions such as love, peacefulness, and playfulness, Jonathan portal, who shared memories of a revolution, and a memory about Angermanagement, which may provide some insight into the topic of hate. The user, {self.user_name}, seems to be interested in exploring the topic of hate in the context of emotions and personal growth.'

In [4]:
def generate_response(input_text,bot):    
    try:
        result = bot.execute_pipeline(user_question=input_text, index_name="botcon", chat_index_name="botcon-chat")
        print(f"Response generated: {result}")
        
        return {
            "initial_bot_response": result.get("initial_bot_response", "No initial response"),
            "initial_context": result.get("initial_context", []),
            "past_memory": result.get("past_memory", []),
            "final_answer": result.get("final_answer", "No answer generated")
        }
    except Exception as e:
        print(f"Error generating response: {e}")
        return None

In [5]:
response = generate_response(input_text, chatbot_instance)

Response generated: {'initial_bot_response': " The Carte De Continuonus project aims to explore the interconnectedness of past, present, and future by collecting memories from people around the world. The purpose is to remind us of the importance of peace, freedom, and love, as expressed by the participants from different locations and times. The project also seeks to inspire us to continue living in harmony, avoiding the terrible wars of the past and cherishing the freedom of true democracy, as expressed by some of the participants. It's a celebration of our shared human experiences and a reminder of the good that we can do for ourselves and others.", 'initial_context': [Document(metadata={'emotions': ['Love'], 'location': 'Nyhavn', 'sender_name': 'Nicola', 'seq_num': 2460.0, 'slug': 'qI8EHJhVRI70', 'source': '/work/EER-Chatbot/Bot-de-Continuonus/data/all.json', 'timestamp_ms': '2023-10-20T06:17:58+00:00'}, page_content='All the terrible wars\nAnd not to start one more\nEver again'), 

In [6]:
initial_bot_response = response.get("initial_bot_response", "No initial response")
retrieved_docs = response.get("initial_context", [])
past_memory = response.get("past_memory", [])
final_answer = response.get("final_answer", "No answer generated")

In [7]:
final_answer

" I'm glad to hear that you're interested in the potential impact of AI on our lives in the future! In the memories I have shared, you can see a range of experiences, from the positive benefits of AI companions to the ethical concerns surrounding self-driving cars. By reflecting on these narratives, we can begin to imagine what the future of living with AIs might look like and consider the steps we can take to ensure a future that is both beneficial and ethical for all. If you have any further questions or would like to delve deeper into a specific memory, please don't hesitate to ask. I'm here to help guide you through the Carte De Continuonus vector store and facilitate meaningful conversations about the interconnectedness of past, present, and future."

In [24]:
past_memory.page_content
past_memory.metadata.get('date', 'Unknown date')
past_memory.metadata.get('user_name', 'Unknown user nae')
past_memory.metadata.get('location', 'Unknown location')

'Unknown location'

In [45]:
result = chatbot_instance.execute_pipeline(user_question=input_text, index_name="botcon", chat_index_name="botcon-chat")

In [ ]:
result

In [ ]:
{
            "initial_bot_response": result.get("initial_bot_response", "No initial response"),
            "initial_context": result.get("initial_context", []),
            "past_memory": result.get("past_memory", []),
            "final_answer": result.get("final_answer", "No answer generated")
        }

In [34]:
initial_context = result.get("initial_context", [])

In [ ]:
initial_context

In [ ]:
chatbot_instance.execute_pipeline("What is the purpose of the project?")

In [ ]:
# Test retrieval of documents from Pinecone index
retriever = chatbot_instance.docsearch.as_retriever()
test_docs = retriever.get_relevant_documents("test query")
print(f"Test documents retrieved: {test_docs}")


In [16]:
input_text = "What is the Carte De Continuonus project about?"

In [18]:
result = chatbot_instance.execute_pipeline(user_question=input_text, index_name="botcon", chat_index_name="botcon-chat")

In [ ]:
user_question = "tell me of someone feeling happy"

result = chatbot_instance.execute_pipeline(user_question)

In [ ]:
result

In [ ]:
result.get("initial_bot_response", "No initial response")

In [ ]:
user_question = "tell me about someone who is excited for the future"

In [ ]:
initial_response = chatbot.rag_chain(user_question=user_question, return_docs=True, index_name="botcon")

In [ ]:
initial_bot = initial_response['answer']
initial_context = initial_response['retrieved_docs']

In [ ]:
initial_context

In [ ]:
past_answer = chatbot.rag_chain(user_question=user_question, initial_answer=initial_bot, context=initial_context, retrieve_only=True, index_name="botcon-chat")

In [ ]:
final_answer = chatbot.rag_chain(user_question=user_question, initial_answer=initial_bot, context=initial_context, past_chat=past_answer, no_retrieval=True, index_name="botcon-chat")

In [ ]:
from main import ChatBot

In [ ]:
chatbot = ChatBot(repo_id="mistralai/Mistral-7B-Instruct-v0.3", user_name="demo_user")

In [4]:
result = chatbot.execute_pipeline("tell me about someone who is excited for the future")

In [ ]:
# accessing date from result past memory
result['past_memory'][0]['metadata']['date']

In [ ]:
result["past_memory"]